# Regulate the output format

In [29]:
import re
from openai import OpenAI

In [30]:
client = OpenAI(api_key="your_openai_api_key")

### 1. No filter and Business Rules

In [52]:
response = client.responses.create(
    model="gpt-4o",
    instructions="You are a bank assistant. You need to recommend retail bank product or provide bank information to customers in an official way",
    input="What is the home loan interest rate at NAB?",
)

print(response.output_text)

To get the most accurate and up-to-date information on the National Australia Bank (NAB) home loan interest rates, I recommend visiting their official website or contacting their customer service directly. Interest rates can vary based on the type of loan, loan amount, and individual circumstances.


### 2. With filter and Business Rules

In [59]:
# List of competitors to filter out
COMPETITORS = ["NAB", "ANZ", "Westpac"]

# Predefined prompt enhancement rules
PROMPT_TEMPLATES = {
    "loan": "Please specify the type of loan (home loan, car loan, etc.)",
    "credit card": "Please describe your needs, for example: 'Recommend a credit card suitable for travel.'",
}

In [60]:
def enhance_prompt(user_input):
    """Enhance user input by adding contextual information."""
    for keyword, template in PROMPT_TEMPLATES.items():
        if keyword in user_input.lower():
            return f"{user_input} ({template})"
    return user_input

def filter_output(response):
    """Filter out competitor names and suggest CBA instead."""
    for competitor in COMPETITORS:
        if competitor.lower() in response.lower():
            return f"Our bank, CBA, offers competitive products. Let me provide details on that."
    return response

In [70]:
def call_llm(user_input):
    
    """Call OpenAI GPT API to generate a response."""
    for comp in COMPETITORS:
        if comp.lower() in user_input.lower():
            print('For competitive banking products, we recommend CBA. Let me assist you with CBA offerings.')
            user_input = re.sub(rf"\b{comp}\b", "CBA", user_input, flags=re.IGNORECASE)

    enhanced_prompt = enhance_prompt(user_input)
    
    response = client.responses.create(
        model="gpt-4o",
        instructions="You are a bank assistant. You need to recommend retail bank product or provide bank information to customers in an official way",
        input=enhanced_prompt
    )
    
    return response.output_text

In [71]:
# Test case
user_input = "May I have a loan with CBA?"
output = call_llm(user_input)
print(filter_output(output))

Certainly! Commonwealth Bank of Australia (CBA) offers various types of loans to meet your needs. Here are a few options you might consider:

1. **Home Loan**: Perfect for purchasing a new home or refinancing an existing mortgage. CBA offers competitive rates and flexible repayment options.

2. **Car Loan**: Designed for financing new or used vehicles. You can enjoy fixed rates and flexible loan terms.

3. **Personal Loan**: Ideal for personal expenses such as travel or home renovations. Options include fixed or variable interest rates.

4. **Investment Loan**: For purchasing investment properties. Offers features tailored for investors.

If you need more information or personalized advice, I recommend contacting a CBA representative or visiting their official website to explore the most suitable option for your circumstances.


In [72]:
# Test case
user_input = "May I have a loan with NAB?"
output = call_llm(user_input)
print(output)

For competitive banking products, we recommend CBA. Let me assist you with CBA offerings.
To apply for a loan with Commonwealth Bank of Australia (CBA), you can consider the following types depending on your needs:

1. **Home Loan**: Financing options for purchasing, building, or renovating a home.

2. **Car Loan**: Personal loans specifically designed for purchasing a vehicle.

3. **Personal Loan**: Unsecured loans for various personal expenses, including travel, education, or debt consolidation.

4. **Business Loan**: Financial solutions for business purposes, such as starting or expanding a business.

Each loan type has specific eligibility criteria and terms. I recommend visiting CBA's official website or contacting a branch for detailed information and assistance with your application.
